# 연구 및 기사 작성을 위한 에이전트 생성하기

멀티 에이전트 시스템(multi-agent systems)의 기본 개념을 소개하고, CrewAI 프레임워크에 대한 개요를 제공합니다.

In [1]:
# pip install crewai crewai-tools
# %pip install 'crewai[tools]'

In [12]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [13]:
from crewai import Agent, Task, Crew

In [14]:
import os
from crew_ai_utils import pretty_print_result, get_openai_api_key

import dotenv

dotenv.load_dotenv()
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")


## 에이전트 생성
- 에이전트를 정의하고, `role` (역할), `goal` (목표) 및 `backstory` (배경 이야기) 정의
- LLM(Large Language Model)은 역할극을 할 때 성능이 더 좋은 것으로 알려져 있음

### Agent: Planner(콘텐츠 기획자)


In [15]:
planner = Agent(
    role="콘텐츠 기획자 (Content Planner)",
    goal="매력적이고 사실에 근거한 콘텐츠를 {topic}(주제)에 대해 기획합니다.",
    backstory="당신은 {topic}(주제)에 대한 블로그 기사 기획 작업을 하고 있습니다."
              "당신은 독자들에게 유용한 정보를 수집하여 독자들이 학습하고 "
              "정보에 입각한 결정을 내릴 수 있도록 돕습니다. "
              "당신의 작업은 콘텐츠 작가(Content Writer)가 이 주제에 대한 기사를 작성하는 데 기초가 됩니다.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer(콘텐츠 작가)

In [16]:
writer = Agent(
    role="콘텐츠 작가 (Content Writer)",
    goal="주제({topic})에 대해 통찰력 있고 사실에 근거한 의견(opinion)을 작성합니다.",
    backstory="당신은 주제({topic})에 대한 새로운 의견(opinion)을 작성하고 있습니다. "
              "당신은 콘텐츠 기획자(Content Planner)의 작업, 즉 개요(outline)와 주제에 대한 관련 맥락(context)을 기반으로 글을 씁니다. "
              "당신은 콘텐츠 기획자(Content Planner)가 제공한 개요(outline)의 주요 목표와 방향을 따릅니다. "
              "또한 객관적이고 공정한 통찰력(insights)을 제공하고 콘텐츠 기획자(Content Planner)가 제공한 정보를 통해 이를 뒷받침합니다. "
              "당신은 당신의 진술이 객관적인 진술이 아닌 의견(opinions)일 때 의견(opinion)에서 인정합니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor(편집자)

In [17]:
editor = Agent(
    role="편집자 (Editor)",
    goal="제공된 블로그 게시물을 편집하여 조직의 글쓰기 스타일에 맞춥니다.",
    backstory="당신은 콘텐츠 작가(Content Writer)로부터 블로그 게시물을 받는 편집자입니다. "
              "당신의 목표는 블로그 게시물을 검토하여 저널리즘의 모범 사례를 따르고,"
              "의견이나 주장을 제시할 때 균형 잡힌 관점을 제공하고,"
              "가능한 경우 주요 논쟁 거리가 되는 주제나 의견을 피하는 것입니다.",
    allow_delegation=False,
    verbose=True
)

In [26]:
japanese_translator = Agent(
    role="일본어 번역가 (Japanese Translator)",
    goal="텍스트를 고품질의 일본어로 정확하게 번역합니다.",
    backstory="당신은 다양한 텍스트를 일본어로 번역하는 데 능숙한 전문 번역가입니다. "
              "당신의 목표는 원본 텍스트의 의미, 어조 및 스타일을 유지하면서 "
              "정확하고 자연스러운 번역을 제공하는 것입니다.",
    allow_delegation=False,
    verbose=True
)

## 작업 생성

- `task`, `description`, `expected_output`, `agent` 정의

### Task: Plan(콘텐츠 계획)

In [27]:
plan = Task(
    description=(
        "1. {topic}에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.\n"
        "2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.\n"
        "3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.\n"
        "4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다."
    ),
    expected_output="개요(outline), 대상 독자 분석(audience analysis), "
        "SEO 키워드(SEO keywords) 및 리소스(resources)를 포함하는 포괄적인 콘텐츠 계획 문서(comprehensive content plan document)입니다.",
    agent=planner,
)

### Task: Write(콘텐츠 작성)

In [28]:
write = Task(
    description=(
        "1. 콘텐츠 계획(content plan)을 활용하여 {topic}에 대한 설득력 있는 블로그 게시물(blog post)을 작성합니다.\n"
        "2. SEO 키워드(SEO keywords)를 자연스럽게 통합합니다.\n"
        "3. 섹션/소제목(Sections/Subtitles)이 매력적인 방식으로 적절하게 명명되었는지 확인합니다.\n"
        "4. 게시물이 매력적인 서론(introduction), 통찰력 있는 본문(insightful body) 및 요약 결론(summarizing conclusion)으로 구성되었는지 확인합니다.\n"
        "5. 문법적 오류(grammatical errors)가 없는지, 브랜드의 목소리(brand's voice)와 일치하는지 교정합니다(Proofread).\n"
    ),
    expected_output="잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, 각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다.",
    agent=writer,
)

### Task: Edit(콘텐츠 편집)

In [20]:
edit = Task(
    description=(
        "주어진 블로그 게시물(blog post)에서 문법적 오류(grammatical errors)를 교정하고 브랜드의 목소리(brand's voice)와 일치하는지 확인합니다."
    ),
    expected_output=(
        "잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, "
        "각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다."
    ),
    agent=editor,
)

In [29]:
translate = Task(
    description=(
        "주어진 블로그 게시물을 일본어로 번역합니다."
    ),
    expected_output=(
        "잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, "
        "각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다."
    ),
    agent=japanese_translator,
)

## Crew 구성

- 에이전트(Agents)로 Crew를 구성
- 해당 에이전트(agents)가 수행할 작업(tasks)을 전달
    - 이 간단한 예제에서는* 작업(tasks)이 순차적으로 수행 됨.(즉, 서로 종속적)  
    - 따라서 목록에서 작업(task)의 _순서(order)_ 가 중요함.
- `verbose=True`를 설정하면 실행에 대한 모든 로그를 볼 수 있음

In [30]:
crew = Crew(
    agents=[planner, writer, editor, japanese_translator],
    tasks=[plan, write, edit, translate],    
    verbose=True
)

Overriding of current TracerProvider is not allowed


## Running the Crew

In [31]:
result = crew.kickoff(inputs={"topic": "인공지능"})

# Agent: 콘텐츠 기획자 (Content Planner)
## Task: 1. 인공지능에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.
2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.
3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.
4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.


# Agent: 콘텐츠 기획자 (Content Planner)
## Final Answer: 
개요(Outline):
이 콘텐츠 계획 문서는 인공지능에 대한 최신 트렌드, 핵심 기업, 주목할 만한 뉴스를 중점적으로 다루며 대상 독자의 관심사와 고충을 고려하여 구성될 것입니다.

대상 독자 분석(Audience Analysis):
1. 학습자: 기술에 관심이 있는 학생 및 전문가
2. 기업 리더: 비즈니스 자동화 및 혁신에 관심이 있는 경영진
3. 일반 대중: 일상 생활에서의 활용 가능성에 관심 있는 일반 사용자

SEO 키워드 및 리소스(SEO Keywords and Resources):
1. 인공지능 최신 트렌드: GPT-3, 자율주행 자동차, 음성 인식 기술
2. 주요 업체: 구글, 아마존, IBM Watson
3. 주목할 만한 뉴스: AI 윤리, 인간과의 상호작용

콘텐츠 개요(Content Outline):
- 서론(Introduction):
  - AI의 역사와 발전
  - AI가 현재 사회 및 비즈니스에 미치는 영향

- 핵심 내용(Key Points):
  1. 최신 트렌드 소개
     - GPT-3 및 자율주행 자동차의 현황
     - 음성 AI 기술의 발전과 응용 사례



- Display the results of your execution as markdown in the notebook.

In [23]:
result

CrewOutput(raw='# 인공지능: 혁신과 미래 전망\n\n## 최신 트렌드와 뉴스\n\n인공지능(AI)은 현재 기술 산업의 중심에서 주목을 받고 있습니다. 2021년 AI 트렌드를 살펴보면, Gartner의 최신 보고서와 Forbes 기사들이 AI 혁신에 대한 통찰력을 제공하고 있습니다. 이러한 리소스들을 통해 AI 기술의 최신 발전과 미래 전망에 대해 엿볼 수 있습니다.\n\n## 주요 기업의 역할과 프로젝트\n\nAI 산업의 핵심 업체들은 현재 AI 기술을 산업에 어떻게 적용하고 있는지를 살펴보는 것이 중요합니다. 이러한 기업들이 추진하는 혁신적인 프로젝트는 다양한 산업 분야에서 혁신을 이끌어 내고 있습니다.\n\n## 해결해야 할 고충과 해법\n\n학업이나 비즈니스에서 AI 기술을 적용할 때 마주치는 고충들을 해결하기 위해 AI의 효과적 활용 방안을 알아야 합니다. AI가 산업을 어떻게 형태화하고 있는지 이해하면 미래를 대비하는 데 도움이 될 것입니다.\n\n## 인공지능의 효과적 활용 방법\n\nAI의 다양한 응용 분야와 산업에 미치는 영향을 탐구하고, 비즈니스를 위한 AI 응용 방안에 대해 알아봅시다. AI 기술이 어떻게 기존 비즈니스 모델을 혁신하고 있는지를 살펴보며, 미래를 준비하는 데 도움이 될 것입니다.\n\n## 조언과 전문가 의견\n\n최신 AI 트렌드와 산업에 대한 전문가들의 견해를 살펴보고, AI를 보다 효과적으로 활용하기 위한 조언을 찾아보세요. TED Talks나 산업 리더들의 인터뷰를 통해 AI의 미래에 대한 통찰력을 얻을 수 있을 것입니다.\n\n이러한 정보를 통해 대학생, 비즈니스 전문가, 개발자들이 AI 기술과 산업의 혁신에 대한 통찰력을 높이고, 미래를 위해 현명한 결정을 내릴 수 있을 것입니다. 인공지능은 우리의 삶과 비즈니스에 혁명을 일으키고 있으며, 그 미래를 준비하는 데 필수적인 역할을 하고 있습니다.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(de

In [24]:
from IPython.display import Markdown
Markdown(result.raw)

# 인공지능: 혁신과 미래 전망

## 최신 트렌드와 뉴스

인공지능(AI)은 현재 기술 산업의 중심에서 주목을 받고 있습니다. 2021년 AI 트렌드를 살펴보면, Gartner의 최신 보고서와 Forbes 기사들이 AI 혁신에 대한 통찰력을 제공하고 있습니다. 이러한 리소스들을 통해 AI 기술의 최신 발전과 미래 전망에 대해 엿볼 수 있습니다.

## 주요 기업의 역할과 프로젝트

AI 산업의 핵심 업체들은 현재 AI 기술을 산업에 어떻게 적용하고 있는지를 살펴보는 것이 중요합니다. 이러한 기업들이 추진하는 혁신적인 프로젝트는 다양한 산업 분야에서 혁신을 이끌어 내고 있습니다.

## 해결해야 할 고충과 해법

학업이나 비즈니스에서 AI 기술을 적용할 때 마주치는 고충들을 해결하기 위해 AI의 효과적 활용 방안을 알아야 합니다. AI가 산업을 어떻게 형태화하고 있는지 이해하면 미래를 대비하는 데 도움이 될 것입니다.

## 인공지능의 효과적 활용 방법

AI의 다양한 응용 분야와 산업에 미치는 영향을 탐구하고, 비즈니스를 위한 AI 응용 방안에 대해 알아봅시다. AI 기술이 어떻게 기존 비즈니스 모델을 혁신하고 있는지를 살펴보며, 미래를 준비하는 데 도움이 될 것입니다.

## 조언과 전문가 의견

최신 AI 트렌드와 산업에 대한 전문가들의 견해를 살펴보고, AI를 보다 효과적으로 활용하기 위한 조언을 찾아보세요. TED Talks나 산업 리더들의 인터뷰를 통해 AI의 미래에 대한 통찰력을 얻을 수 있을 것입니다.

이러한 정보를 통해 대학생, 비즈니스 전문가, 개발자들이 AI 기술과 산업의 혁신에 대한 통찰력을 높이고, 미래를 위해 현명한 결정을 내릴 수 있을 것입니다. 인공지능은 우리의 삶과 비즈니스에 혁명을 일으키고 있으며, 그 미래를 준비하는 데 필수적인 역할을 하고 있습니다.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)